# 1. polglot-ko-1.3b를 squarelike/sharegpt_deepl_ko_translation로 파인 튜닝한 모델

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "aeolian83/Gugugo_for_DnD_v0.7"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

In [2]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [3]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

stop_words = ["</끝>"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [4]:
def gen(lan="en", x=""):
    if (lan == "ko"):
        prompt = f"### 한국어: {x}</끝>\n### 영어:"
    else:
        prompt = f"### 영어: {x}</끝>\n### 한국어:"
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=2048,
        temperature=0.001,
        no_repeat_ngram_size=10,
        early_stopping=True,
        eos_token_id=2,
        stopping_criteria=stopping_criteria
    )
    return tokenizer.decode(gened[0]).replace(prompt+" ", "")

In [5]:
gen(lan="en", x="The Steward informed me that a mysterious package has somehow arrived on my ship. I should speak with her to learn more.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/aeolian83/anaconda3/envs/translateDnD/lib/python3.10/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


'집사가 내게 신비한 꾸러미가 내 배에 도착했다고 알려주었다. 그녀와 더 많은 것을 배우기 위해 그녀와 이야기를 해봐야겠다.</끝>'

In [6]:
gen(lan="en", x="While you are wearing armor, you gain a +1 bonus to AC.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'갑옷을 입고 있는 동안에는 AC에 +1의 보너스를 얻습니다.</끝>'

In [7]:
gen(lan="en", x="Chultans are the human natives of Chult. They have dark skin, black hair, and a rich culture. Driven out of the jungle by monsters and undead, they took refuge behind the stout walls of Port Nyanzaru, gave up their dynasties, and united to become a mercantile power in the southern ocean, eager to do business with anyone who visits their perilous land. A few Chultans migrated northward and established outposts in dis tant cities such as Athkatla, Baldur's Gate, Cali mport, Memnon, and Zazesspur.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'컬트는 컬트의 인간들입니다. 그들은 검은 피부, 검은 머리카락, 풍부한 문화를 가지고 있습니다. 괴물과 언데드에 의해 사막을 방황하던 그들은 포니족의 벽 뒤에 피난처를 만들어, 포니족의 왕족들을 쫓아내고, 남부 해안의 항구 니야자루의 거대한 벽 뒤에 피난처를 만들어, 그들은 자신들의 위험한 땅을 방문하는 사람들에게 사업을 하기를 원하는, 위험한 땅을 방문하는 사람들을 위한 상인으로서의 힘을 가진 자들로 가득 차 있습니다. 몇몇의 컬트들이 북쪽으로 이주하여 아스카틀라, 발더스 게이트, 칼림나, 엠노, 그리고 자지뿌러 같은 도시들에 정착했습니다.</끝>'

In [11]:
gen(lan="en", x="The bell tower rises another 10 feet above the thatched roof of the bailey, making it 30 feet tall overa ll. In a normal day, the bell is ru ng twelve times : it's run g every four hours to s ignal the change of watch, and it's rung 10 minutes before each of those to signal that a change of watch is about to occur. At the change of the watch, it 's rung a number of times equaling the number of the watch: once at the start of the first watch, twice at the start of the second watch, and so forth. On what the garrison calls \"the wake-ups,\" it rings once.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/aeolian83/anaconda3/envs/translateDnD/lib/python3.10/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


'### 영어: The bell tower rises another 10 feet above the thatched roof of the bailey, making it 30 feet tall overa ll. In a normal day, the bell is ru ng twelve times : it\'s run g every four hours to s ignal the change of watch, and it\'s rung 10 minutes before each of those to signal that a change of watch is about to occur. At the change of the watch, it\'s rung a number of times equaling the number of the watch: once at the start of the first watch, twice at the start of the second watch, and so forth. On what the garrison calls "the wake-ups," it rings once.</끝>\n### 한국어: 벨 탑은 울타리에 매달린 지붕의 30피트 위에 세워져 있으며, 30피트 높이로 30피트나 높게 솟아 있다. 보통의 날에는 벨이 12시간 동안 작동하며, 그것은 시계의 변화를 알리기 위해 매 4시간마다 작동한다. 시계의 변화에 따라 시계의 수는 \'초시계\'와 같다. 첫 번째 시계의 시작 시간부터 시계의 시작 시간까지는 두 번째 시계의 시작 시간까지는 두 번째의 시계의 시작 시간까지, 시계의 수는 시계의 시작 시간에 따라 결정된다. 수비대가 말하는 \'해탈\'은 초시계의 첫 번째 시간에 한 번 울리는 것이다.</끝>'

In [12]:
gen(lan="en", x="Nanny Pu'pu, a green hag, plays the part of an ancient crone to the hilt. She claims that a ll the other villagers were killed over the course of many years by winged creatures that live on the south side of the plateau.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'녹색의 망아지, 나니푸는, 녹색 망아지가 손에 든 망치로 옛 거인의 역할을 하고 있다. 그녀는 남쪽에 있는 언덕에 살고 있는 날개 달린 생명체들에 의해 수천 년 동안 많은 사람들이 죽었다고 주장한다.</끝>'

In [13]:
gen(lan="en", x="Erik, I've gone in search of the nine shrines. V.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'에릭, 아홉 개의 사원을 찾으러 왔소. V.</끝>'